In [17]:
import csv 
import math

print('starting to train..')

label_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/train.label'
data_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/train.data'
number_of_class=20;
number_of_words=61188

# construct label dict
f = open(label_path)
reader = csv.reader(f,delimiter=" ")
label={}
i=0;
for rows in reader:
    label[i]=int(rows[0])-1
    i+=1
    
num_of_doc=len(label)

# Nk - number of doc per class
Nk=[0]*number_of_class;
for i in label:
    Nk[label[i]]=Nk[label[i]]+1;


min_count=1
# #twpc total nuber of words per class
twpc = [0] * number_of_class
# #fwpc freq words per class
fwpc = [[min_count] * number_of_words for i in range(number_of_class)] 

f = open(data_path)
reader = csv.reader(f,delimiter=" ")
for rows in reader:
    twpc[label[int(rows[0])-1]]+=int(rows[2])
    fwpc[label[int(rows[0])-1]][int(rows[1])-1]+=int(rows[2])
        

smooth=15# smothing value

PrWgivenC = [[0.0] * number_of_words for i in range(number_of_class)]  
for c in range(0,number_of_class):
    for w in range(0,number_of_words):
#         PrWgivenC[c][w]=math.log(nk[c][w])-math.log(Nk[c])+10
        PrWgivenC[c][w]=math.log(fwpc[c][w])-math.log(twpc[c])+smooth
        
PrC=[0]*number_of_class
for c in range(0,number_of_class):
    PrC[c]=math.log(Nk[c])-math.log(num_of_doc)
#     PrC[c]=Nk[c]/num_of_doc

def getClass(wc):
    global PrC,PrWgivenC
    PrCgivenW=[0.0]*number_of_words
    sum_pr=0.0;
    for c in range(0,number_of_class):
#         pr=1.0;
        pr=0.0;
        for w in range(0,number_of_words):
#             pr=pr*math.pow(PrWgivenC[c][w],wc[w])
            pr=pr+PrWgivenC[c][w]*wc[w]
        PrCgivenW[c]=pr+PrC[c]
        sum_pr+=PrCgivenW[c]
    if sum_pr==0:
        return -1
#     select arg max
    max_c=0
    for c in range(0,number_of_class):
        PrCgivenW[c]=PrCgivenW[c]/sum_pr
        if PrCgivenW[max_c]<PrCgivenW[c]:
            max_c=c
    return max_c;

print("done!")


starting to train..
done!


In [14]:
print('starting to buid wc for test')
test_label_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/test.label'
test_data_path = '/Users/apple/Desktop/workspace/20news-bydate/matlab/test.data'
f = open(test_label_path)
reader = csv.reader(f,delimiter=" ")
tlabel={}
i=0;
for rows in reader:
    tlabel[i]=int(rows[0])-1
    i+=1

no_of_test_doc=7505

wc = [[0] * number_of_words for i in range(no_of_test_doc)] 

f = open(test_data_path)
reader = csv.reader(f,delimiter=" ")
for rows in reader:
    wc[int(rows[0])-1][int(rows[1])-1]+=int(rows[2])

print("created wc")

starting to buid wc for test
created wc


In [21]:
print("starting to test")
f = open(test_label_path)
reader = csv.reader(f,delimiter=" ")
tlabel={}
i=0;
for rows in reader:
    tlabel[i]=int(rows[0])-1
    i+=1
    
correct=0
incorrect=0
for i in range(0,no_of_test_doc):
    if(i%1000==0):
        print("-- doc index:"+str(i))
    l=getClass(wc[i])
    if l==tlabel[i]:
        correct+=1
    else:
        incorrect+=1

print("final result: c: "+str(correct)+"  e: "+str(incorrect))
print("Accuracy:"+str(correct/(correct+incorrect)))

starting to test
-- doc index:0
-- doc index:1000
-- doc index:2000
-- doc index:3000
-- doc index:4000
-- doc index:5000
-- doc index:6000
-- doc index:7000
final result: c: 5836  e: 1669
Accuracy:0.7776149233844104
